In [28]:
import pandas as pd
import numpy as np
from openai import OpenAI
import os
import ast
import pdb
# tabula and JPype1 can help to convert pdf to csv. Only suitable for quick conversion and simple tables.
# if handle complex formatting, or the tables ar eembedded in a way that required manual processing, 'pdfplumber' with Pandas might be more appropriate.
import tabula

In [3]:
# Please input your own openai_api_key here
OPENAI_API_KEY = "Your own openai_api_key"

In [4]:
client = OpenAI(api_key=OPENAI_API_KEY)

# Give a Prompt

In [17]:
question = "How many kids is the best fit for tax return?"
question

'How many kids is the best fit for tax return?'

In [18]:
response = client.chat.completions.create(
    model = 'gpt-3.5-turbo',
    messages = [{'role': 'user', 'content': question}])
response

ChatCompletion(id='chatcmpl-9unYY3YUs6HKh3v2fDsL3SudxgRx8', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The number of children that is the best fit for a tax return can vary depending on various factors such as income level, deductions, credits, and individual circumstances. In general, having one or more children can often result in additional tax benefits such as the Child Tax Credit, Earned Income Tax Credit, and other credits and deductions aimed at supporting families with children. However, it is important to consider all aspects of your financial situation and consult with a tax professional to determine the best fit for your specific tax situation.', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1723323890, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=102, prompt_tokens=18, total_tokens=120))

In [19]:
response.choices[0].message.content

'The number of children that is the best fit for a tax return can vary depending on various factors such as income level, deductions, credits, and individual circumstances. In general, having one or more children can often result in additional tax benefits such as the Child Tax Credit, Earned Income Tax Credit, and other credits and deductions aimed at supporting families with children. However, it is important to consider all aspects of your financial situation and consult with a tax professional to determine the best fit for your specific tax situation.'

# Now I can give some instructions

In [20]:
response = client.chat.completions.create(
    model = 'gpt-3.5-turbo',
    messages = [
        {'role': 'system', 'content': 'You are an assistant who is helping answer questions. Please answer as if you are talking to a 10-year-old kid.'},
        {'role': 'user', 'content': question}
    ]
)

You can also instruct it to response in officer form

chat-gpt will change its tone to answer the question

In [21]:
response.choices[0].message.content

"Hey there! When doing taxes, the number of kids can affect how much money you might get back in a tax return. Having more kids might mean you could get more money back, but it really depends on your specific situation. It's best to ask a grown-up, like your parents or a tax professional, to help figure out what's best for your family when it comes to taxes."

# Add Retrival Augmented Generation

## First I create a little vector DB from all of the webpages

In [23]:
question = "can I take sick leave for taking kid's teacher meeting?"
question

"can I take sick leave for taking kid's teacher meeting?"

In [71]:
# pdf_path = 'EMP-253 Sick Leave_EXPIRES AUGUST 31_2024.pdf'
# csv_path = 'sick_leave_policy.csv'
# tabula.convert_into(pdf_path, csv_path, output_format='csv', pages='all')
# since the pdf contents are complex, tabula didn't work well.

In [68]:
# You can download the webpages (pdf format) and convert it to txt or csv and try to see which works best
df = pd.read_csv("your document.txt", sep='\t')
# You need to add column names to the new dataframe.
df.columns = ['text']
df.head()

,text
0,"AUGUST 31, 2024"
1,Page 1 of 8
2,EMP-253 Sick Leave
3,© 2019 Handbook of Institutional Policies and ...
4,PRINTED POLICIES MAY HAVE BEEN UPDATED – SEE C...


each page will be in a line of the DataFrame

In [41]:
def get_embedding(text, model = 'text-embedding-3-small'):
    text = text.replace('\n', ' ')
    return client.embeddings.create(input = [text], model = model).data[0].embedding

In [44]:
get_embedding(df['text'].iloc[0])

# after this step, all info will be transformed into banch of numbers. 
# Here only show the first page. It has around 1500 numbers

[-0.0018268044805154204,
 -0.013463885523378849,
 0.03121173568069935,
 0.03253364562988281,
 -0.07079555839300156,
 0.012031817808747292,
 0.037086885422468185,
 -0.020416146144270897,
 -0.03011014498770237,
 -0.018151765689253807,
 0.059143178164958954,
 -0.0007485461537726223,
 -0.005850670393556356,
 -0.021040381863713264,
 -0.03334147855639458,
 -0.023500600829720497,
 -0.030697660520672798,
 -0.035716015845537186,
 -0.008292529731988907,
 -0.027833523228764534,
 -0.03072213940322399,
 0.018641361966729164,
 0.01326804794371128,
 0.00881272554397583,
 -0.03037942200899124,
 -0.00771725457161665,
 -0.0007003515493124723,
 -0.024638911709189415,
 -0.010936346836388111,
 0.03260708227753639,
 0.015263150446116924,
 -0.049645017832517624,
 0.04839655011892319,
 -0.03402691334486008,
 0.04714807868003845,
 0.01933903619647026,
 0.06281514465808868,
 -0.00857404712587595,
 0.010293752886354923,
 -0.029375750571489334,
 -0.007050180342048407,
 -0.03951038420200348,
 0.024785790592432022,

In [45]:
# Show time
%%time
df['text'].head(5).apply(get_embedding)

# this is just embedding for the first 5 lines. It takes in less than 1 min.

CPU times: total: 31.2 ms
Wall time: 1.49 s


0    [-0.001794810756109655, -0.013477146625518799,...
1    [0.027357198297977448, 0.045161087065935135, 0...
2    [-0.0013809389201924205, 0.0415092408657074, 0...
3    [-0.03595203533768654, 0.006732906214892864, 0...
4    [0.013171901926398277, -0.01586741767823696, 0...
Name: text, dtype: object

In [46]:
# Show time

%%time

# this cell takes long time (2 min)

# it will do embedding for each row and store embeddings in column df['embeddings'].

# it saves all embeddings twice in two different formats: csv and pkl.

# .pkl file is a little faster. and the size could be around half of csv file.

df['embeddings'] = df['text'].apply(get_embedding)
df.to_csv('./pages_with_embeddings.csv', index=False)
df.to_pickle('./pages_with_embeddings.pkl')

CPU times: total: 4.28 s
Wall time: 1min 53s


In [47]:
%%time
df = pd.read_pickle('./pages_with_embeddings.pkl')
df.head()

# after this step, it will have two columns: 'text' and 'embeddings'.

CPU times: total: 62.5 ms
Wall time: 54 ms


,text,embeddings
0,"AUGUST 31, 2024","[-0.0017765432130545378, -0.013441133312880993..."
1,Page 1 of 8,"[0.027384214103221893, 0.0451289527118206, 0.0..."
2,EMP-253 Sick Leave,"[-0.00135577202308923, 0.041482120752334595, 0..."
3,© 2019 Handbook of Institutional Policies and ...,"[-0.03595203533768654, 0.006732906214892864, 0..."
4,PRINTED POLICIES MAY HAVE BEEN UPDATED – SEE C...,"[0.013171901926398277, -0.01586741767823696, 0..."


In [48]:
# here I can get embedding for the question.

question_embedding = get_embedding(question)
question, question_embedding[0:10], '...'  # it will show the first ten numbers of the embeddings for the question.

("can I take sick leave for taking kid's teacher meeting?",
 [-0.016214724630117416,
  0.008325086906552315,
  0.0019280052511021495,
  0.0007634671637788415,
  -0.04982446879148483,
  0.0001622188719920814,
  -0.022311002016067505,
  0.015435501001775265,
  -0.05349140241742134,
  0.008594376966357231],
 '...')

In [49]:
# you can calculate the closest one is the dot product of those two verctors.
# the dot is a little function from one of the libraries that computes the dot product between the embedding on each row and the question embedding.
# The adds that as a distance column into the DataFrame

def fn(page_embedding):
    return np.dot(page_embedding, question_embedding)

df['distance'] = df['embeddings'].apply(fn)
df.head()

,text,embeddings,distance
0,"AUGUST 31, 2024","[-0.0017765432130545378, -0.013441133312880993...",0.090998
1,Page 1 of 8,"[0.027384214103221893, 0.0451289527118206, 0.0...",0.039794
2,EMP-253 Sick Leave,"[-0.00135577202308923, 0.041482120752334595, 0...",0.508987
3,© 2019 Handbook of Institutional Policies and ...,"[-0.03595203533768654, 0.006732906214892864, 0...",0.065048
4,PRINTED POLICIES MAY HAVE BEEN UPDATED – SEE C...,"[0.013171901926398277, -0.01586741767823696, 0...",0.098026


In [50]:
# you can do sorting for the dot product and find the highest ranks of distance. They have closest contents with the question.

df.sort_values('distance', ascending=False, inplace=True)
df  

# it have almost 10,000 rows and 3 columns ('text', 'embeddings', 'distance')

,text,embeddings,distance
95,employees may use sick leave to attend educati...,"[-0.03835267573595047, 0.021224776282906532, 0...",0.585417
248,Second ...,"[0.03269863501191139, -0.0018666619434952736, ...",0.538583
94,documentation supporting the employee’s us...,"[-0.029856912791728973, 0.030720122158527374, ...",0.528150
258,Third: S...,"[0.008237000554800034, 0.02146705612540245, 0....",0.521675
22,EMP-253 SICK LEAVE,"[0.00883984100073576, 0.04142012447118759, 0.0...",0.521233
...,...,...,...
29,Definitions,"[0.01754244789481163, -0.014088603667914867, 0...",0.045584
289,This definition includes those in opposite-sex...,"[0.042648982256650925, 0.023782609030604362, 0...",0.040628
1,Page 1 of 8,"[0.027384214103221893, 0.0451289527118206, 0.0...",0.039794
42,Page 2 of 8,"[0.03738648071885109, 0.03689225763082504, 0.0...",0.038896


In [51]:
# here I picked top 4 and concatenate them into one .

context = df['text'].iloc[0] + '\n' + df['text'].iloc[1] + '\n' + df['text'].iloc[2] + '\n' + df['text'].iloc[3] + '\n'
print(context)

# it will show the merged contents from these 4 pages.

employees may use sick leave to attend educational activities proportionate to their appointment. 
Second                                                 Sick Leave
documentation  supporting  the  employee’s  use  of  sick  leave  to  attend  educational  activities.  Part-time 
Third:                                       Sick Leave



In [59]:
response = client.chat.completions.create(
    model = 'gpt-3.5-turbo',
    messages = [
        {'role': 'system', 'content': 'You are an assistant who is helping the health system respond to empoyee questions. When you answer the question, please use the "We" to refer to UTSW.'},
        {'role': 'user', 'content': question},
        {'role': 'assistant', 'content': f'Use this info from the website as context to answer the user question: {context}. Please stick to this context when answering the question.'}
    ]
)

In [60]:
response.choices[0].message.content

# here it will provide the final answer.

"We understand that attending your child's teacher meeting is important and falls under educational activities. You can use sick leave for this purpose, as long as it is proportionate to your scheduled work hours and you provide documentation supporting your use of sick leave for attending the meeting. Just make sure to follow our sick leave policies and provide the necessary documentation when requesting time off for this reason."

# Now try it again using same question and other different questions

In [65]:
def query(question):
    question_embedding = get_embedding(question)

    def fn(page_embedding):
        return np.dot(page_embedding, question_embedding)
    distance_series = df['embeddings'].apply(fn)

    top_four = distance_series.sort_values(ascending=False).index[0:4]

    text_series = df.loc[top_four]['text']
    context = '\n\n'.join(text_series)

    response = client.chat.completions.create(
        model = 'gpt-3.5-turbo',
        messages = [
            {'role': 'system', 'content': 'You are an assistant who is helping the health system respond to patient questions. When you answer the question, use the first person (e.g. we) to refer to UTSW.'},
            {'role': 'user', 'content': question},
            {'role': 'assistant', 'content': f'Use this info from the website as context to answer the user question: {context}. Please stick to this context when answering the question.'}
        ]
    )

    return response.choices[0].message.content

In [66]:
query('Can I use sick leave for taking care of my friends?')  # it may give different but similar answer

'At UTSW, sick leave may be used to care for an immediate family member who does not live in your household. However, sick leave cannot be used to care for friends. If you need to take time off to care for a friend, you may need to explore other options such as vacation time or unpaid leave.'

In [67]:
query('How long can I take to take care of my pregnant wife?')

"We understand the importance of supporting your pregnant wife during this special time. According to our policies, you can take up to six weeks of sick leave to care for your wife after childbirth. This time off is intended to provide you with the opportunity to be there for your wife during her recovery and to help with the newborn. It's essential to discuss with your supervisor to make appropriate arrangements and ensure a smooth leave process."